In [1]:
%run ../processing/read_data.py
%run ../processing/functions.ipynb

import numpy as np
import pandas as pd
import os
from PIL import Image

In [2]:
path_to_read = '../../data/interpolate_data_low_quality/'
files = []

for r, d, f in os.walk(path_to_read):
    for file in f:
        if '.png' in file:
            files.append(os.path.join(r, file))
files.sort()
np_files = np.asarray(files)

In [3]:
#Verification if each transient has 4 curves
temp = 'SN000017'
count = 0
for f in np_files:
    if (count == 4):
        temp = f[40:48]
        count = 0
    if ((temp == f[40:48]) & (count < 4)):
        count += 1
    elif ((temp != f[40:48]) & (count < 4)):
        print(f)

In [28]:
#set(types) -> {'IIL', 'IIP', 'II_', 'IIn', 'Ia_', 'Ib_', 'Ibc', 'Ic_'}
conversion = {'IIL' : 5, 'IIP' : 7, 'II_' : 4, 'IIn' : 6, 'Ia_' : 0, 'Ib_' : 1, 'Ibc' : 2, 'Ic_' : 3}

In [29]:
np_grouped = np_files.reshape(round(np_files.shape[0]/4), 4)
class_id = []
#CHANGE HERE THE INDEX EVERYTIME WE CHANGE THE NAME FROM THE PATH OF THE FOLDER
for g in np_grouped:
    class_id = class_id + [[g[0][40:48], g[0][49:52]]] 
types = []
for ids in class_id:
    types = types + [conversion[ids[1]]]
    
astr_objs = []
for i in range(len(np_grouped)):
    astr_objs = astr_objs + [class_id[i]]
    for file in np_grouped[i]:
        img = Image.open(file).convert('L')
        arr = np.array(img)
        astr_objs[-1] = astr_objs[-1] + [arr[5:35,10:56]] 
        #CHANGE HERE IF WE EVENTUALLY CHANGE THE SIZE OF THE IMAGE, OR WANT TO CUT OTHER FRAMES

print( "The quantity of objects is: ", len(astr_objs))

The quantity of objects is:  21317


In [30]:
labels = []
images = []
ids = []
for o in astr_objs:
    labels.append((conversion[o[1]])) #converting here to numbers to assure label in training
    images.append(o[2:])
    ids.append(o[0])
labels = np.asarray(labels)
images = np.asarray(images)
ids = np.asarray(ids)

train_n = 18000

train_labels = labels[0:train_n]
test_labels = labels[train_n:]

train_images = images[0:train_n]
test_images = images[train_n:]

train_ids = ids[0:train_n]
test_ids = ids[train_n:]


In [41]:
print('The shapes of the objects are:')
print('ids: ', ids.shape)
print('labels: ', labels.shape)
print('images: ', images.shape)
print('train_images: ', train_images.shape)
print('Other useful object: "test_images"')

The shapes of the objects are:
ids:  (21317,)
labels:  (21317,)
images:  (21317, 4, 30, 46)
train_images:  (18000, 4, 30, 46)
Other useful object: "test_images"


In [32]:
train_images = np.around(train_images/255, decimals = 2)
test_images = np.around(test_images/255, decimals = 2)

In [35]:
from __future__ import absolute_import, division, print_function
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.examples.tutorials.mnist import input_data
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

1.13.1
